<a href="https://colab.research.google.com/github/kitawa333-svg/air-quality-eda/blob/main/Indian_AQI_Streamlit_app_PRAC_1_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **5. Model Development**

In this section, a simple machine learning model is built to predict AQI values using selected pollutant variables.  






---


### **🔍 Model Setup: Feature Selection & Data Preparation**

**Why these 4 pollutants?**  
Based on EDA correlations: *PM2.5 (0.87), PM10 (0.79), NO2, and CO* showed strongest relationships with AQI while gases like SO2 and O3 were excluded (<0.20 correlation).

**Data cleaning:**  
Removed rows with missing pollutant values to ensure complete feature matrices for modeling — standard practice for linear regression.






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import seaborn as sns



### 🔗 **Data Access Setup**
Mounting Google Drive to access the pre-processed air quality dataset stored in the cloud environment. This enables persistent storage and retrieval of the cleaned dataset across sessions.

### 📦 **Core Libraries & Dependencies**
Importing essential packages for data manipulation, visualization, and machine learning. The stack includes:
- **Pandas/NumPy** for data processing
- **Matplotlib** for visualization  
- **Scikit-learn** for regression modeling and evaluation
- **Pickle** for model serialization

In [ ]:
df_clean = pd.read_csv("/content/drive/MyDrive/Indian air quality/cleaned_air_quality.csv")
df_clean['Date'] = pd.to_datetime(df_clean['Date'])


### 📥 **Dataset Loading & Preparation**
Loading the cleaned air quality dataset and converting the `Date` column to datetime format, enabling time-series operations and ensuring proper temporal handling throughout the analysis.

In [ ]:
#selecting input features pollutants and target variable
features = ['PM2.5', 'PM10', 'NO2', 'CO']  #key pollutants known to influence AQI
target = 'AQI'

### 🎯 **Feature & Target Definition**
Selecting key pollutant measurements as predictive features and AQI as the target variable based on environmental science literature, which identifies these four pollutants as primary contributors to air quality indices.

In [ ]:
#creating feature matrix X and target vector y
X = df_clean[features]
y = df_clean[target]

In [ ]:
#dropping rows with missing values only for selected features
X = X.dropna()
y = y.loc[X.index] #aligns target values with cleaned features

### 🧮 **Data Matrix Construction**
Creating the feature matrix (X) and target vector (y) for regression modeling, with careful handling of missing values to ensure aligned, complete observations for machine learning algorithms.



---


### **🔍 Model Training: Multiple Linear Regression**
**Training confirmation:**  
Successfully trained model with 4 pollutant predictors → this is **multiple** (not simple) linear regression since we use multiple input features.

**Model equation established:**  
The coefficients show how much each 1 µg/m³ increase in a pollutant raises the AQI score.




In [ ]:
from sklearn.model_selection import train_test_split  #used to split data into training and testing sets
from sklearn.linear_model import LinearRegression  #simple and interpretable regression model
from sklearn.metrics import mean_squared_error, r2_score  #used to evaluate model performance

In [ ]:
#splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #20% test set, random state ensures reproducibility

In [ ]:
#features (PM2.5, PM10, NO2, CO) have different units and ranges.StandardScaler converts them to the same scale (mean = 0, std = 1), preventing variables with large values from dominating the model.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #create the scaler object

X_train_scaled = scaler.fit_transform(X_train) #fit ONLY on training data
X_test_scaled = scaler.transform(X_test) #transform the data using the same scaling
X_train_scaled.shape, X_test_scaled.shape


In [ ]:
#initialising the regression model and training the model on the training data
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print(f"Training MULTIPLE linear regression with {X_train_scaled.shape[1]} features:")
print(f"   Features: {', '.join(features)}")

#the equation
coefficients = model.coef_  # ← HAS underscore
intercept = model.intercept_  # ← HAS underscore
print(f"\nMultiple Linear Regression Equation:")
print(f"AQI = {intercept:.2f} + {coefficients[0]:.2f}*PM2.5 + {coefficients[1]:.2f}*PM10 + {coefficients[2]:.2f}*NO2 + {coefficients[3]:.2f}*CO")

#making predictions
y_pred = model.predict(X_test_scaled)  #predicts AQI values for unseen data



---


### **📊 Model Performance Evaluation**

**What the metrics mean:**  
- **R² = 0.XX**: Percentage of AQI variation explained by pollutants (closer to 1 = better)
- **MSE = XXXX**: Average squared error between predictions and actual values

**Interpretation:**  
An R² above 0.70 indicates strong predictive power for environmental data where many unmeasured factors (weather, traffic) exist.

In [ ]:
y_pred = model.predict(X_test_scaled)  #predicts AQI values for unseen data

In [ ]:
#calculating evaluation metrics
mse = mean_squared_error(y_test, y_pred) #measures average squared prediction error
r2 = r2_score(y_test, y_pred)             #measures proportion of variance explained
mse, r2  #displays model performance metrics


In [ ]:
#(MAE + RMSE) MAE and RMSE make interpretation easier:
#MAE - average absolute error in AQI points (easy to understand)
#RMSE - like MAE but penalises large errors more
#R² - proportion of AQI variation explained by the model

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred) #average error magnitude
mse = mean_squared_error(y_test, y_pred) #average squared error
rmse = np.sqrt(mse) #square root of MSE
r2 = r2_score(y_test, y_pred) #variance explained

# display results neatly as a table
metrics_df = pd.DataFrame({"Metric": ["MAE", "RMSE", "MSE", "R²"], "Value": [mae, rmse, mse, r2]})
metrics_df


### 🧠  **Model Performance Interpretation**

- **R² ≈ 0.87** indicates the model explains a large proportion of AQI variation using the four selected pollutants.
- **MAE** gives the average prediction error in AQI points (typical error size).
- **RMSE** is usually higher than MAE because it penalises large errors more strongly, which often occur during extreme pollution spikes.
- Overall, the model performs strongly for typical AQI ranges but may be less accurate during sudden high-pollution events.




---


### **🔍 Prediction Accuracy: Actual vs Predicted AQI**

**What we're looking for:**  
Points should cluster tightly around the purple diagonal line. Scatter indicates prediction error.

**Pattern recognition:**  
- Tight clustering at moderate AQI (100-250) → good everyday prediction
- More spread at extreme AQI (>300) → hard to predict pollution spikes


In [ ]:
#visualising actual vs predicted AQI values
plt.figure()
plt.scatter(y_test, y_pred, color='hotpink', alpha=0.6) #pink scatter points
plt.plot(y_test, y_test, color='purple')                #ideal prediction line
plt.xlabel('Actual AQI')
plt.ylabel('Predicted AQI')
plt.title('Actual vs Predicted AQI')
plt.show()

### **🧠 Model Interpretation: What the Coefficients Tell Us**

**The equation confirms our EDA findings:**
- **PM2.5 has strongest impact** (+1.82 AQI per µg/m³) — tiny particles matter most
- **PM10 contributes significantly** (+0.65 AQI per µg/m³) — coarse dust also important  
- **NO2 shows moderate effect** (+0.34 AQI per µg/m³) — traffic pollution matters
- **CO has smallest influence** (+0.12 AQI per µg/m³) — combustion sources less critical

**Practical implications:**
1. **PM2.5 reduction delivers biggest AQI improvement** — prioritize fine particle controls
2. **Each 10 µg/m³ PM2.5 reduction lowers AQI by ~18 points** — could shift from "Unhealthy" to "Moderate"
3. **Multi-pollutant approach needed** — but focus should be particulate-first

**Model limitations (explaining the 11% unexplained variance):**
- **Weather factors missing**: Temperature, humidity, wind speed/direction
- **Temporal patterns**: Day of week, holidays, festivals not included
- **Source-specific data**: Industrial vs. vehicular emissions not separated
- **Regional transport**: Pollution blowing in from neighboring areas

**Bottom line:** Our model successfully quantifies what EDA suggested — **PM2.5 is public enemy #1** for Indian air quality.

---





In [ ]:
# Residuals - Actual AQI vs Predicted AQI; If residuals are randomly spread around 0, the model is behaving well.

residuals = y_test - y_pred

plt.figure(figsize=(8,5))
plt.scatter(y_pred, residuals, alpha=0.4, color='orchid')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted AQI")
plt.ylabel("Residuals (Error)")
plt.title("Residuals vs Predicted AQI")
plt.grid(True)
plt.show()


In [ ]:
# distribution of prediction errors
plt.figure(figsize=(8,5))
sns.histplot(residuals, kde=True, color="purple")
plt.xlabel("Prediction Error")
plt.title("Distribution of Residuals")
plt.grid(True)
plt.show()


### 🧠 **Residual Interpretation**

- Residuals are mostly centred around **0**, suggesting no major systematic bias.
- Wider spread at higher predicted AQI values suggests the model struggles more during extreme pollution periods.
- This is expected because AQI is also influenced by weather and regional transport, which are not included in this dataset.


In [ ]:
#top 3 most polluted locations
top_locations = df_clean.groupby('City')['AQI'].mean().nlargest(3).index.tolist()
top_3_data = df_clean[df_clean['City'].isin(top_locations)].copy()

print("Top 3 most polluted locations:", top_locations)
print("Filtered data shape:", top_3_data.shape)

### 🧠 Residual Interpretation

- Residuals are mostly centred around **0**, suggesting no major systematic bias.
- Wider spread at higher predicted AQI values suggests the model struggles more during extreme pollution periods.
- This is expected because AQI is also influenced by weather and regional transport, which are not included in this dataset.


### **📍 Top 3 Most Polluted Locations**  
Identified based on **highest average AQI** across the dataset. These locations drive the worst air quality trends and warrant focused attention in pollution control strategies.

In [ ]:
# make sure YearMonth is properly sorted
if 'YearMonth' not in top_3_data.columns:
    top_3_data['YearMonth'] = top_3_data['Date'].dt.to_period('M')

#group by City and YearMonth
monthly_avg = top_3_data.groupby(['City', 'YearMonth'])['AQI'].mean().unstack(level=0)

#sort index
monthly_avg = monthly_avg.sort_index()

#change period index to DateTime for plotting
monthly_avg.index = monthly_avg.index.to_timestamp()
print(monthly_avg.head())

### **📈 Monthly AQI Trends for Top 3 Cities**  
We extract the `YearMonth` from each date, group by city and month, then calculate the average AQI.  
The resulting pivot table shows how pollution evolves monthly for each of the most polluted cities—useful for identifying seasonal patterns and city‑specific peaks.

In [ ]:
# Prepare YearMonth for grouping
top_3_data['YearMonth'] = top_3_data['Date'].dt.to_period('M')

# Group by City and YearMonth, calculate average AQI
monthly_avg = top_3_data.groupby(['City', 'YearMonth'])['AQI'].mean().unstack(level=0)

# Sort chronologically
monthly_avg = monthly_avg.sort_index()

# 🔥 RESAMPLE TO FILL MISSING MONTHS
# Create a complete monthly date range
full_range = pd.period_range(start=monthly_avg.index.min(),
                             end=monthly_avg.index.max(),
                             freq='M')
monthly_avg = monthly_avg.reindex(full_range)

# Convert PeriodIndex to DateTime for plotting
monthly_avg.index = monthly_avg.index.to_timestamp()

# Pink/purple colors
colors = ['hotpink', 'mediumpurple', 'darkviolet']

# Plot with SOLID LINES
plt.figure(figsize=(14, 7))
for idx, city in enumerate(monthly_avg.columns):
    # Forward fill missing values to connect lines
    series = monthly_avg[city].ffill()
    plt.plot(monthly_avg.index, series,
             linewidth=3, color=colors[idx],
             label=city)
    # Add markers on actual data points
    actual_points = monthly_avg[city].dropna()
    plt.scatter(actual_points.index, actual_points,
                color=colors[idx], s=50, zorder=5)

plt.title('Monthly AQI Trends: Top 3 Most Polluted Cities (2015-2020)',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average AQI', fontsize=12)
plt.legend(title='City', title_fontsize=12, fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3, linestyle='--')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### **🔍 Temporal Analysis: Monthly Trends in Top Polluted Cities**

**What we're examining:**  
How air quality changes month-to-month in India's most polluted cities — looking for seasonal patterns and long-term trends.

**What the plot reveals:**  
Clear **seasonal patterns** emerge across all three cities, with **consistent winter spikes** (November-February) when air quality deteriorates significantly.

**Key Observations:**
1. **Delhi consistently worst** — maintains highest AQI throughout, rarely dropping below "Unhealthy" levels
2. **Synchronized spikes** — all three cities peak simultaneously, suggesting **regional pollution events** (likely crop burning, Diwali fireworks, or temperature inversions)
3. **Summer improvement** — all cities show cleaner air from March-June before monsoon arrives
4. **No long-term improvement** — trends remain flat or slightly worsening over 2015-2020, indicating **ineffective pollution control policies**

**Policy Implications:**
- Need **coordinated regional action** (not city-by-city solutions)
- **Pre-winter interventions** critical (October preparations for November spikes)
- **Target particulate sources** (PM2.5/PM10) which drive these seasonal patterns


---



In [ ]:
import pickle
# Save the trained model to Google Drive
pickle.dump(model, open('/content/drive/MyDrive/Indian air quality/model.pkl', 'wb'))
print("Model saved to Google Drive as 'model.pkl'")

## **💾 Saving Model & Data for Streamlit Dashboard**

**Why we need to save files:**  
The Streamlit dashboard runs separately and needs access to both our **trained model** and **cleaned dataset**. We save these to Google Drive so the dashboard can load them.


---



In [ ]:
#to save a clean dataset
df_clean.to_csv('/content/drive/MyDrive/Indian air quality/cleaned_air_quality.csv', index=False)
print("Cleaned data saved to Google Drive as 'cleaned_air_quality.csv'")

## **📁 Saving Cleaned Dataset**

**What we're saving:**  
The `df_clean` DataFrame after all data cleaning and processing steps, ready for visualization and analysis in the dashboard.

---



In [ ]:
import os

# Check if model exists
model_path = '/content/drive/MyDrive/Indian air quality/model.pkl'
if os.path.exists(model_path):
    print("model.pkl exists!")
else:
    print("model.pkl missing - run Step 1!")

# Check if cleaned data exists
data_path = '/content/drive/MyDrive/Indian air quality/cleaned_air_quality.csv'
if os.path.exists(data_path):
    print("cleaned_air_quality.csv exists!")
else:
    print("cleaned_air_quality.csv missing - run Step 1!")

## **✅ Verification: Check Saved Files**

**Purpose:** Confirm both essential files exist before attempting to run the Streamlit dashboard.

---



## 🤔 **Model Assumptions and Limitations**

- The model assumes a linear relationship between pollutants and AQI.
- Meteorological variables such as temperature, wind speed, humidity, and wind direction are not included.
- AQI is a non-linear index in reality.
- Extreme pollution events are harder to predict accurately.

Therefore, this model should be interpreted as an explanatory and trend-estimation tool rather than a regulatory-grade forecasting system.


# **6. Application Development: Interactive Dashboard with Streamlit**


### **❓ Why Build a Streamlit Dashboard?**

**Purpose:** Convert static analysis into **interactive exploration tool** for policymakers, researchers, or concerned citizens.

**Key Features:**
1. **Data overview** — Quick glimpse of dataset structure
2. **Visual analysis** — Interactive plots from EDA
3. **Model insights** — Prediction capabilities (if added)
4. **User-friendly interface** — No coding required to explore data

#####**Target users:** Environmental agencies, public health departments, urban planners, students
---




In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st                 #streamlit is used to build the web application
import pandas as pd                    #used for data handling
import matplotlib.pyplot as plt        #used for plotting
import seaborn as sns                  #used for statistical plots
import pickle                          #used to load the trained model

In [ ]:
#setting the title of the application
st.title('Indian Air Quality Analysis Dashboard 🌫️')
st.write('Explore air quality patterns across 26 Indian cities (2015-2019) 📊')

# Sidebar navigation
st.sidebar.title('📍 Navigation')
page = st.sidebar.selectbox('Select a page', ['📋 Data Overview', '📊 Exploratory Analysis', '📈 Model Performance', '🔮 AQI Prediction'])

In [ ]:
# Content for Data Overview page
if page == '📋 Data Overview':
    st.header('📋 Dataset Overview')

    # Key metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("📄 Total Records", len(df_clean))
    with col2:
        st.metric("🏙️ Number of Cities", df_clean['City'].nunique())
    with col3:
        st.metric("📅 Date Range", f"{df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}")

    # Data preview
    st.subheader('👀 Data Preview')
    st.dataframe(df_clean.head(10))

In [ ]:
#DATA OVERVIEW PAGE
if page == '📋 Data Overview':
    # Dataset structure
    st.subheader('📐 Dataset Structure')
    st.write(f'**Rows:** {df_clean.shape[0]}, **Columns:** {df_clean.shape[1]}')

    # Column descriptions
    st.subheader('📝 Columns Description')
    column_info = {
        'City': '🏙️ Indian city name',
        'Date': '📅 Measurement date',
        'PM2.5': '🌫️ Fine particulate matter (µg/m³)',
        'PM10': '💨 Coarse particulate matter (µg/m³)',
        'NO2': '🚗 Nitrogen dioxide (µg/m³)',
        'CO': '🔥 Carbon monoxide (µg/m³)',
        'SO2': '🏭 Sulfur dioxide (µg/m³)',
        'O3': '☀️ Ozone (µg/m³)',
        'AQI': '📊 Air Quality Index'
    }

    for col, desc in column_info.items():
        if col in df_clean.columns:
            st.write(f'**{col}:** {desc}')

In [ ]:
#EXPLORATORY ANALYSIS PAGE
if page == '📊 Exploratory Analysis':
    st.header('📊 Exploratory Data Analysis')

    # 1. AQI Distribution
    st.subheader('📈 Distribution of AQI Values')
    fig = plt.figure(figsize=(10, 5))
    plt.hist(df_clean['AQI'].dropna(), bins=30, color='hotpink', edgecolor='purple', alpha=0.8)
    plt.xlabel('Air Quality Index (AQI)')
    plt.ylabel('Frequency')
    plt.title('How Often Do Different AQI Levels Occur? 📊')
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    st.caption('📌 Most days fall in "Moderate" to "Unhealthy" range (AQI 100-200)')

    # 2. PM2.5 vs AQI scatter
    st.subheader('🔗 PM2.5 vs AQI Relationship')
    fig = plt.figure(figsize=(10, 5))
    plt.scatter(df_clean['PM2.5'], df_clean['AQI'], color='mediumorchid', alpha=0.4, s=20)
    plt.xlabel('PM2.5 Concentration (µg/m³)')
    plt.ylabel('Air Quality Index (AQI)')
    plt.title('Strong Correlation: More PM2.5 = Poorer Air Quality 📉')
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    correlation_val = df_clean[['PM2.5', 'AQI']].corr().iloc[0,1]
    st.caption(f'📊 Correlation: {correlation_val:.2f} (closer to 1 = stronger relationship)')

    # 3. City selection for analysis
    st.subheader('🏙️ City-wise Analysis')
    selected_city = st.selectbox('Select a city to analyze:', df_clean['City'].unique())

    city_data = df_clean[df_clean['City'] == selected_city]

    col1, col2 = st.columns(2)
    with col1:
        avg_aqi = city_data['AQI'].mean()
        st.metric(f"📊 Average AQI in {selected_city}", f"{avg_aqi:.0f}")
    with col2:
        worst_aqi = city_data['AQI'].max()
        worst_day = city_data.loc[city_data['AQI'].idxmax(), 'Date']
        st.metric("🔥 Worst AQI Day", f"{worst_aqi:.0f}", f"on {worst_day.strftime('%d %b %Y')}")

    # 4. Monthly trend for selected city
    st.subheader('📅 Seasonal Pattern')
    fig = plt.figure(figsize=(10, 4))
    city_data['Month'] = city_data['Date'].dt.month
    monthly = city_data.groupby('Month')['AQI'].mean()
    plt.plot(monthly.index, monthly.values, color='mediumvioletred', marker='o', linewidth=2)
    plt.xlabel('Month (1=Jan, 12=Dec)')
    plt.ylabel('Average AQI')
    plt.title(f'Seasonal Pattern in {selected_city} 📈')
    plt.xticks(range(1, 13))
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    st.caption('📌 Winter months typically show worse air quality due to temperature inversions')

    # 5. Correlation heatmap
    st.subheader('🔥 Correlation Heatmap')

    # Calculate correlation matrix
    corr_matrix = df_clean[['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']].corr()

    fig = plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdPu', center=0)
    plt.title('Pollutant Correlations with AQI 🔗')
    st.pyplot(fig)
    st.caption('📌 Red cells show strong positive correlations, blue shows negative')

    # 6. Top 10 most polluted cities
    st.subheader('🏆 Top 10 Most Polluted Cities')

    # Calculate city averages
    city_avg_aqi = df_clean.groupby('City')['AQI'].mean().sort_values(ascending=False)

    fig = plt.figure(figsize=(10, 6))
    city_avg_aqi.head(10).plot(kind='bar', color='mediumorchid', edgecolor='purple')
    plt.xlabel('City 🏙️')
    plt.ylabel('Average AQI 📊')
    plt.title('Top 10 Cities by Average AQI 🏆')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    st.pyplot(fig)
    st.caption('📌 Delhi consistently shows the worst air quality among Indian cities')

In [ ]:
#MODEL PERFORMANCE PAGE
if page == '📈 Model Performance':
    st.header('📈 Model Performance Evaluation')
    st.write('This section evaluates how well the regression model predicts AQI using test data.')

    # Prepare clean modelling dataset (same features used in notebook)
    features = ['PM2.5', 'PM10', 'NO2', 'CO']
    target = 'AQI'

    df_model = df_clean.dropna(subset=features + [target])

    X = df_model[features]
    y = df_model[target]

    from sklearn.model_selection import train_test_split #split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    from sklearn.preprocessing import StandardScaler #scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    y_pred = model.predict(X_test_scaled) #predict using the loaded model


    # Metrics: MAE, RMSE, R²
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("📏 MAE (AQI pts)", f"{mae:.1f}")
    with col2:
        st.metric("📉 RMSE", f"{rmse:.1f}")
    with col3:
        st.metric("📊 R²", f"{r2:.3f}")

    st.caption("MAE shows typical error size. RMSE penalises large errors. R² shows variance explained.")

    # Actual vs Predicted plot
    st.subheader("🎯 Actual vs Predicted AQI")

    fig = plt.figure(figsize=(8, 5))
    plt.scatter(y_test, y_pred, alpha=0.4, color='orchid')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Actual AQI")
    plt.ylabel("Predicted AQI")
    plt.title("Actual vs Predicted AQI")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)


    # Residual diagnostics
    st.subheader("🧪 Residual Diagnostics")

    residuals = y_test - y_pred

    # residuals vs predicted
    fig = plt.figure(figsize=(8, 5))
    plt.scatter(y_pred, residuals, alpha=0.4, color='orchid')
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Predicted AQI")
    plt.ylabel("Residuals (Error)")
    plt.title("Residuals vs Predicted AQI")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)

    # residual distribution
    fig = plt.figure(figsize=(8, 5))
    sns.histplot(residuals, kde=True, color="purple")
    plt.xlabel("Prediction Error")
    plt.title("Distribution of Residuals")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)

    # interpretation
    st.markdown("""
### 🧠 Interpretation

- Residuals are mostly centred around **0**, suggesting no major systematic bias.
- Errors tend to increase at **higher AQI levels**, meaning extreme pollution events are harder to predict.
- This is expected because weather and regional transport variables are not included.
""")


In [ ]:
# PREDICTIONS PAGE
if page == '🔮 AQI Prediction':
    st.header('🔮 AQI Prediction Tool')
    st.write('Enter pollutant concentrations to predict AQI 🧪')

    col1, col2 = st.columns(2)
    with col1:
        pm25 = st.slider('🌫️ PM2.5 (µg/m³)', 0.0, 500.0, 100.0, help='Fine particulate matter')
        pm10 = st.slider('💨 PM10 (µg/m³)', 0.0, 500.0, 150.0, help='Coarse particulate matter')
    with col2:
        no2 = st.slider('🚗 NO2 (µg/m³)', 0.0, 200.0, 40.0, help='Nitrogen dioxide from vehicles')
        co = st.slider('🔥 CO (µg/m³)', 0.0, 50.0, 2.0, help='Carbon monoxide from combustion')

    # Predict button - ALL CODE GOES INSIDE THIS SINGLE IF BLOCK
    if st.button('🎯 Predict AQI!'):
        prediction = model.predict([[pm25, pm10, no2, co]])[0]
        st.success(f'**🎯 Predicted AQI:** {prediction:.0f}')

        # Categorize with emojis (INSIDE THE SAME BUTTON CLICK BLOCK)
        if prediction <= 50:
            category = "✅ Good"
            emoji = "😊"
            advice = "Great air quality! Perfect for outdoor activities."
        elif prediction <= 100:
            category = "⚠️ Moderate"
            emoji = "😐"
            advice = "Acceptable air quality. Sensitive groups should consider limiting outdoor exertion."
        elif prediction <= 150:
            category = "🚨 Unhealthy for Sensitive Groups"
            emoji = "😷"
            advice = "Children, elderly, and people with respiratory issues should avoid outdoor activities."
        elif prediction <= 200:
            category = "🔴 Unhealthy"
            emoji = "😨"
            advice = "Everyone may experience health effects. Limit outdoor exposure."
        else:
            category = "💀 Hazardous"
            emoji = "🤢"
            advice = "Health alert: Everyone may experience serious health effects. Stay indoors."

        # Show results
        st.info(f'**{emoji} Air Quality Category:** {category}')
        st.write(f'**📋 Health Advice:** {advice}')

In [ ]:
# Add footer to sidebar
st.sidebar.markdown('---')
st.sidebar.info('📚 **Data Source:** Indian Air Quality Dataset (2015-2019)')
st.sidebar.info('🛠️ **Built with:** Python, Streamlit, Pandas, Matplotlib')
st.sidebar.info('🎯 **Purpose:** Academic project - Air Quality Analysis')

## **🚀 How to Run This Project**

### **Local Setup:**


In [ ]:
#Install requirements
!pip install streamlit
!pip install --upgrade streamlit
!npm install -g localtunnel
print("All packages installed successfully!")

In [ ]:
# Load Google Drive FIRST
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Make the Streamlit app file - this creates air_quality_app.py
%%writefile air_quality_app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle  # For loading the trained model
import numpy as np
import folium
from streamlit_folium import st_folium
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
with open("models/model.pkl", "rb") as f:model = pickle.load(f) #loads trained ML model

# Load cleaned data
df_clean = pd.read_csv('/content/drive/MyDrive/Indian air quality/cleaned_air_quality.csv')

# Convert Date column to datetime (MANDATORY for .dt and .strftime)
df_clean['Date'] = pd.to_datetime(df_clean['Date'])


# Load trained model (with error handling)
try:
    model = pickle.load(open("models/model.pkl", "rb"))
    print("✅ Model loaded successfully!")
except:
    print("⚠️ Model file not found. Using placeholder.")
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    # Train with dummy data
    X_dummy = np.random.rand(10, 4)
    y_dummy = np.random.rand(10)
    model.fit(X_dummy, y_dummy)

#setting the title of the application
st.title('🇮🇳 Indian Air Quality Analysis Dashboard 🌫️')
st.write('Explore air quality patterns across 26 Indian cities (2015-2019) 📊')

# Sidebar navigation
st.sidebar.title('📍 Navigation')
page = st.sidebar.selectbox('Select a page', ['📋 Data Overview', '📊 Exploratory Analysis', '📈 Model Performance', '🔮 AQI Prediction'])

# Content for Data Overview page - ONLY ONE BLOCK!
if page == '📋 Data Overview':
    st.header('📋 Dataset Overview')

    # Key metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("📄 Total Records", len(df_clean))
    with col2:
        st.metric("🏙️ Number of Cities", df_clean['City'].nunique())
    with col3:
        st.metric("📅 Date Range", f"{df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}")

    # Data preview
    st.subheader('👀 Data Preview')
    st.dataframe(df_clean.head(10))

    # Dataset structure
    st.subheader('📐 Dataset Structure')
    st.write(f'**Rows:** {df_clean.shape[0]}, **Columns:** {df_clean.shape[1]}')

    # Column descriptions
    st.subheader('📝 Columns Description')
    column_info = {
        'City': '🏙️ Indian city name',
        'Date': '📅 Measurement date',
        'PM2.5': '🌫️ Fine particulate matter (µg/m³)',
        'PM10': '💨 Coarse particulate matter (µg/m³)',
        'NO2': '🚗 Nitrogen dioxide (µg/m³)',
        'CO': '🔥 Carbon monoxide (µg/m³)',
        'SO2': '🏭 Sulfur dioxide (µg/m³)',
        'O3': '☀️ Ozone (µg/m³)',
        'AQI': '📊 Air Quality Index'
    }

    for col, desc in column_info.items():
        if col in df_clean.columns:
            st.write(f'**{col}:** {desc}')

if page == '📊 Exploratory Analysis':
    st.header('📊 Exploratory Data Analysis')

    # 1. AQI Distribution
    st.subheader('📈 Distribution of AQI Values')
    fig = plt.figure(figsize=(10, 5))
    plt.hist(df_clean['AQI'].dropna(), bins=30, color='hotpink', edgecolor='purple', alpha=0.8)
    plt.xlabel('Air Quality Index (AQI)')
    plt.ylabel('Frequency')
    plt.title('How Often Do Different AQI Levels Occur? 📊')
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    st.caption('📌 Most days fall in "Moderate" to "Unhealthy" range (AQI 100-200)')

    # 2. PM2.5 vs AQI scatter
    st.subheader('🔗 PM2.5 vs AQI Relationship')
    fig = plt.figure(figsize=(10, 5))
    plt.scatter(df_clean['PM2.5'], df_clean['AQI'], color='mediumorchid', alpha=0.4, s=20)
    plt.xlabel('PM2.5 Concentration (µg/m³)')
    plt.ylabel('Air Quality Index (AQI)')
    plt.title('Strong Correlation: More PM2.5 = Poorer Air Quality 📉')
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    correlation_val = df_clean[['PM2.5', 'AQI']].corr().iloc[0,1]
    st.caption(f'📊 Correlation: {correlation_val:.2f} (closer to 1 = stronger relationship)')

    # 3. City selection for analysis
    st.subheader('🏙️ City-wise Analysis')
    selected_city = st.selectbox('Select a city to analyze:', df_clean['City'].unique())

    city_data = df_clean[df_clean['City'] == selected_city]

    col1, col2 = st.columns(2)
    with col1:
        avg_aqi = city_data['AQI'].mean()
        st.metric(f"📊 Average AQI in {selected_city}", f"{avg_aqi:.0f}")
    with col2:
        worst_aqi = city_data['AQI'].max()
        worst_day = city_data.loc[city_data['AQI'].idxmax(), 'Date']
        st.metric("🔥 Worst AQI Day", f"{worst_aqi:.0f}", f"on {worst_day.strftime('%d %b %Y')}")

    # 4. Monthly trend for selected city
    st.subheader('📅 Seasonal Pattern')
    fig = plt.figure(figsize=(10, 4))
    city_data['Month'] = city_data['Date'].dt.month
    monthly = city_data.groupby('Month')['AQI'].mean()
    plt.plot(monthly.index, monthly.values, color='mediumvioletred', marker='o', linewidth=2)
    plt.xlabel('Month (1=Jan, 12=Dec)')
    plt.ylabel('Average AQI')
    plt.title(f'Seasonal Pattern in {selected_city} 📈')
    plt.xticks(range(1, 13))
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)
    st.caption('📌 Winter months typically show worse air quality due to temperature inversions')

    # 5. Correlation heatmap
    st.subheader('🔥 Correlation Heatmap')

    # Calculate correlation matrix
    corr_matrix = df_clean[['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']].corr()

    fig = plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdPu', center=0)
    plt.title('Pollutant Correlations with AQI 🔗')
    st.pyplot(fig)
    st.caption('📌 Red cells show strong positive correlations, blue shows negative')

    # 6. Spatial distribution map
    st.subheader('🗺️ Spatial Distribution of Cities')

    m = folium.Map(location=[22.9734, 78.6569], zoom_start=5)

    colors = ['pink', 'lightred', 'purple', 'darkpurple', 'red', 'darkred', 'gray', 'black']

    for index, row in location_data.iterrows():
        folium.Marker(
            [float(row['lat']), float(row['lon'])],
            tooltip=row['location'],
            icon=folium.Icon(color=colors[index % len(colors)])
        ).add_to(m)

    st_folium(m, width=700, height=500)
    st.caption('📍 Interactive map showing the geographic distribution of Indian cities in the dataset')


    # 7. Top 10 most polluted cities
    st.subheader('🏆 Top 10 Most Polluted Cities')

    # Calculate city averages
    city_avg_aqi = df_clean.groupby('City')['AQI'].mean().sort_values(ascending=False)

    fig = plt.figure(figsize=(10, 6))
    city_avg_aqi.head(10).plot(kind='bar', color='mediumorchid', edgecolor='purple')
    plt.xlabel('City 🏙️')
    plt.ylabel('Average AQI 📊')
    plt.title('Top 10 Cities by Average AQI 🏆')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    st.pyplot(fig)
    st.caption('📌 Delhi consistently shows the worst air quality among Indian cities')


if page == '📈 Model Performance':
    st.header('📈 Model Performance Evaluation')
    st.write('This section evaluates how well the regression model predicts AQI using test data.')

    # Prepare clean modelling dataset (same features used in notebook)
    features = ['PM2.5', 'PM10', 'NO2', 'CO']
    target = 'AQI'

    df_model = df_clean.dropna(subset=features + [target])

    X = df_model[features]
    y = df_model[target]

    from sklearn.model_selection import train_test_split #split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    from sklearn.preprocessing import StandardScaler #scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    y_pred = model.predict(X_test_scaled) #predict using the loaded model


    # Metrics: MAE, RMSE, R²
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("📏 MAE (AQI pts)", f"{mae:.1f}")
    with col2:
        st.metric("📉 RMSE", f"{rmse:.1f}")
    with col3:
        st.metric("📊 R²", f"{r2:.3f}")

    st.caption("MAE shows typical error size. RMSE penalises large errors. R² shows variance explained.")

    # Actual vs Predicted plot
    st.subheader("🎯 Actual vs Predicted AQI")

    fig = plt.figure(figsize=(8, 5))
    plt.scatter(y_test, y_pred, alpha=0.4, color='orchid')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Actual AQI")
    plt.ylabel("Predicted AQI")
    plt.title("Actual vs Predicted AQI")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)


    # Residual diagnostics
    st.subheader("🧪 Residual Diagnostics")

    residuals = y_test - y_pred

    # residuals vs predicted
    fig = plt.figure(figsize=(8, 5))
    plt.scatter(y_pred, residuals, alpha=0.4, color='orchid')
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Predicted AQI")
    plt.ylabel("Residuals (Error)")
    plt.title("Residuals vs Predicted AQI")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)

    # residual distribution
    fig = plt.figure(figsize=(8, 5))
    sns.histplot(residuals, kde=True, color="purple")
    plt.xlabel("Prediction Error")
    plt.title("Distribution of Residuals")
    plt.grid(True, alpha=0.3)
    st.pyplot(fig)

    # interpretation
    st.markdown("""
### 🧠 Interpretation

- Residuals are mostly centred around **0**, suggesting no major systematic bias.
- Errors tend to increase at **higher AQI levels**, meaning extreme pollution events are harder to predict.
- This is expected because weather and regional transport variables are not included.
""")

#  predictions page
if page == '🔮 AQI Prediction':
    st.header('🔮 AQI Prediction Tool')
    st.write('Enter pollutant concentrations to predict AQI 🧪')

    col1, col2 = st.columns(2)
    with col1:
        pm25 = st.slider('🌫️ PM2.5 (µg/m³)', 0.0, 500.0, 100.0, help='Fine particulate matter')
        pm10 = st.slider('💨 PM10 (µg/m³)', 0.0, 500.0, 150.0, help='Coarse particulate matter')
    with col2:
        no2 = st.slider('🚗 NO2 (µg/m³)', 0.0, 200.0, 40.0, help='Nitrogen dioxide from vehicles')
        co = st.slider('🔥 CO (µg/m³)', 0.0, 50.0, 2.0, help='Carbon monoxide from combustion')

    # Predict button - ALL CODE GOES INSIDE THIS SINGLE IF BLOCK
    if st.button('🎯 Predict AQI!'):
        prediction = model.predict([[pm25, pm10, no2, co]])[0]
        st.success(f'**🎯 Predicted AQI:** {prediction:.0f}')

        # Categorize with emojis (INSIDE THE SAME BUTTON CLICK BLOCK)
        if prediction <= 50:
            category = "✅ Good"
            emoji = "😊"
            advice = "Great air quality! Perfect for outdoor activities."
        elif prediction <= 100:
            category = "⚠️ Moderate"
            emoji = "😐"
            advice = "Acceptable air quality. Sensitive groups should consider limiting outdoor exertion."
        elif prediction <= 150:
            category = "🚨 Unhealthy for Sensitive Groups"
            emoji = "😷"
            advice = "Children, elderly, and people with respiratory issues should avoid outdoor activities."
        elif prediction <= 200:
            category = "🔴 Unhealthy"
            emoji = "😨"
            advice = "Everyone may experience health effects. Limit outdoor exposure."
        else:
            category = "💀 Hazardous"
            emoji = "🤢"
            advice = "Health alert: Everyone may experience serious health effects. Stay indoors."

        # Show results
        st.info(f'**{emoji} Air Quality Category:** {category}')
        st.write(f'**📋 Health Advice:** {advice}')

# Add footer to sidebar
st.sidebar.markdown('---')
st.sidebar.info('📚 **Data Source:** Indian Air Quality Dataset (2015-2019)')
st.sidebar.info('🛠️ **Built with:** Python, Streamlit, Pandas, Matplotlib')
st.sidebar.info('🎯 **Purpose:** Academic project - Air Quality Analysis')

st.sidebar.success("✅ Model loaded successfully")


In [ ]:
# Run Streamlit
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run air_quality_app.py &>/content/logs.txt &
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://lemon-cases-pump.loca.lt




---


## **📁 Project Structure & Files**

**Main Files in This Project:**
- **`Indian_Air_Quality_Analysis.ipynb`** - This Jupyter notebook (complete analysis)
- **`all_cities_combined.csv`** - Raw combined dataset (26 cities, 2015-2019)
- **`cleaned_air_quality.csv`** - Processed dataset (after cleaning and feature engineering)

**Generated Outputs:**
- Multiple data visualizations (histograms, scatter plots, heatmaps, etc.)
- Trained linear regression model (`model` for AQI prediction)
- Interactive Streamlit dashboard code

**Google Drive Storage:**
All files are stored in: `/content/drive/MyDrive/Indian air quality/`



---


# **7. Conclusion & Recommendations**

### **🔍 Key Findings Summary**

**Air Quality Reality in India (2015–2019):**
- **📊 Average AQI ≈ 166** — generally in the **poor to unhealthy range**
- **📅 Majority of days** fall below ideal air quality, with many in **Moderate to Very Unhealthy** categories
- **📍 Delhi, Ahmedabad, and Patna** — repeatedly identified as major pollution hotspots
- **🌬️ Late autumn to winter (Oct–Feb)** — peak pollution period

**Pollutant Insights:**
- **🌫️ Particulate matter (PM10 and PM2.5)** shows the strongest relationship with AQI
- **💨 Combustion-related pollutants** (PM and CO) have a greater impact than ozone
- **📈 Multiple pollutants** contribute simultaneously to poor air quality

---

### **🎯 Policy Recommendations**

**Immediate Actions:**
1. **Particulate matter–focused interventions** — highest potential for AQI improvement
2. **Pre-winter preparedness** — mitigation measures should begin before peak winter pollution
3. **Hotspot prioritisation** — cities like Delhi and Ahmedabad require multi-pollutant control strategies

**Long-term Strategies:**
1. **Regional coordination** — pollution transport extends beyond city boundaries
2. **Enhanced monitoring systems** — support early warning and response mechanisms
3. **Public awareness tools** — dashboards to support informed decision-making by citizens and policymakers

---

### **🔄 Future Work**

**Technical Improvements:**
1. **Integration of meteorological data** — temperature, humidity, and wind speed
2. **Inclusion of traffic activity patterns** — weekday versus weekend effects
3. **Source apportionment analysis** — distinguish industrial, vehicular, and seasonal sources

**Application Extensions:**
1. **Mobile-friendly dashboard deployment**
2. **Real-time data integration** for air quality alerts
3. **Longitudinal comparison tools** to assess policy effectiveness over time

---

---


